## Introdução

Aqui irei realizar a etapa de Feature Engineering na base de dados `pos_cash_balance`.

Esta etapa está dividida em kernels diferentes devido o consumo de memória para carregar e manipular as bases.

Os procedimentos realizados nas outras bases podem ser encontrados nos arquivos `02_feat_eng_<nome-da-base>.ipynb`

O objetivo desta etapa consiste, principalmente, em criar variáveis (`book de variáveis`). Ao criar novas variáveis com base nas variáveis existentes, é possível capturar informações adicionais que podem não estar explicitamente presentes nos dados originais.

Devido o volume de dados, optei por utilizar o PySpark em conjunto com o SparkSQL para as operações a seguir.

* Sobre os Dados

A base `pos_cash_balance` possui informações sobre o histórico de pagamentos de POS (Point of Sale) ou empréstimos em dinheiro.

Segundo os Metadados disponibilizados, essas são as informações contidas aqui:

`SK_ID_PREV`: ID do crédito anterior. (Um crédito em nossa amostra pode ter 0, 1, 2 ou mais créditos anteriores no Crédito Habitacional). Será utilizada como PK para conectarmos à base `previous_application`, conforme a esquematização abaixo.

`SK_ID_CURR`: ID do empréstimo.

`MONTHS_BALANCE`: Mês do saldo em relação à data de aplicação (-1 significa a informação mais recente, 0 significa a informação na aplicação - frequentemente será o mesmo que -1, já que muitos bancos não atualizam regularmente as informações no Bureau de Crédito).

`CNT_INSTALMENT`: Prazo do crédito anterior (pode variar ao longo do tempo).

`CNT_INSTALMENT_FUTURE`: Parcelas restantes a pagar no crédito anterior.

`NAME_CONTRACT_STATUS`: Status do contrato durante o mês.

`SK_DPD`: DPD (dias em atraso) durante o mês do crédito anterior.

`SK_DPD_DEF`: DPD durante o mês com tolerância (dívidas com baixos valores de empréstimo são ignoradas) do crédito anterior.

## Utils

* Importando as bibliotecas que irei utilizar

In [1]:
import os
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import when,min, max, sum, round, col, median
spark = SparkSession.builder \
    .appName("FeatureEng") \
    .config("spark.executor.memory", "14g") \
    .config("spark.driver.memory", "14g") \
    .getOrCreate()
from warnings import filterwarnings
filterwarnings('ignore')

## Feature Engineering - POS Cash Balance

### 1. Carregamento e Informações Gerais

* Carregando e Visualizando os dados utilizando o PySpark

In [2]:
pos_cash = spark.read.csv('./DATASETS/POS_CASH_balance.csv', header= True, inferSchema= True)

In [3]:
pos_cash.show(n=3, truncate= False)

+----------+----------+--------------+--------------+---------------------+--------------------+------+----------+
|SK_ID_PREV|SK_ID_CURR|MONTHS_BALANCE|CNT_INSTALMENT|CNT_INSTALMENT_FUTURE|NAME_CONTRACT_STATUS|SK_DPD|SK_DPD_DEF|
+----------+----------+--------------+--------------+---------------------+--------------------+------+----------+
|1803195   |182943    |-31           |48.0          |45.0                 |Active              |0     |0         |
|1715348   |367990    |-33           |36.0          |35.0                 |Active              |0     |0         |
|1784872   |397406    |-32           |12.0          |9.0                  |Active              |0     |0         |
+----------+----------+--------------+--------------+---------------------+--------------------+------+----------+
only showing top 3 rows



* Verificando as dimensões (Linhas x Colunas)

In [4]:
(pos_cash.count(), len(pos_cash.columns))

(10001358, 8)

* Qtd. de Linhas após agrupamento (Para verificar ao final do processo)

In [5]:
pos_cash.groupBy("SK_ID_PREV").count().count()

936325

### 2. Criação de Flags Temporais

In [3]:
# Criando uma View
pos_cash.createOrReplaceTempView('pos_cash')

temp01 = spark.sql("""
SELECT
    *,
        CASE
            WHEN MONTHS_BALANCE >= -3 THEN 1
        ELSE 0
    END AS FL_U3M,
        CASE
            WHEN MONTHS_BALANCE >= -6 THEN 1
        ELSE 0
    END AS FL_U6M,
        CASE
            WHEN MONTHS_BALANCE >= -9 THEN 1
        ELSE 0
    END AS FL_U9M,
        CASE
            WHEN MONTHS_BALANCE >= -12 THEN 1
        ELSE 0
    END AS FL_U12M
FROM
    pos_cash
ORDER BY
    `SK_ID_PREV`;
""")

temp01.createOrReplaceTempView('temp01')

In [7]:
temp01.count()

10001358

### 3. Criação das Flags das Variáveis Categóricas

* Criando uma flag para cada valor único de cada variável categórica

In [4]:
# Lista para armazenar as colunas criadas
flags = []

# Pegando todas as colunas categoricas da tabela
cat_cols = [cat_col[0] for cat_col in temp01.dtypes if cat_col[1] == 'string']

# Gerando as colunas Flag
for cat_col in cat_cols:
    unique_vals = [col[0] for col in temp01.select(cat_col).distinct().collect()]

    for unique_val in unique_vals:
        flags.append(when(col(cat_col) == unique_val, 1).otherwise(0).alias(f'FL_CAT_{cat_col}_{unique_val.upper().replace(" ", "_")}'))

* Realizando o Unpacking e Criando a Tabela

In [5]:
# temp02 = temp01.select("*", *flags)
temp02 = temp01

### 4. Criação das Variáveis (Agrupadas)

* Filtrando as colunas que serão agregadas

In [6]:
# Selecionando as variáveis que serão agregadas (exceto Flags e IDs)
agg_cols = [col for col in temp02.columns if ("FL_" not in col) & ("SK_ID" not in col)]

# Removendo a Coluna de Janela Temporal
agg_cols.remove('MONTHS_BALANCE')

# Removendo as colunas categóricas que tiveram flags criadas
for cat_col in cat_cols:
    agg_cols.remove(cat_col)

* Criando as novas variáveis e agrupando os dados

In [7]:
flags_temporais = ['FL_U3M', 'FL_U6M', 'FL_U9M','FL_U12M']

flags_categoricas = [col for col in temp02.columns if 'FL_CAT_' in col]

#### 4.1 Usando Apenas Flags Temporais

In [8]:
new_cols = []

for flag_temp in flags_temporais:
    nome_flag_temp_corrigido = flag_temp.replace('FL_','')

    for agg_col in agg_cols:

        if 'DPD' in agg_col:
            new_cols.append(round(max(when(col(flag_temp) == 1, col(agg_col))),2).alias(f"QT_MAX_{agg_col}_{nome_flag_temp_corrigido}_POSCASH"))
            new_cols.append(round(min(when(col(flag_temp) == 1, col(agg_col))),2).alias(f"QT_MIN_{agg_col}_{nome_flag_temp_corrigido}_POSCASH"))
        else:
            new_cols.append(round(max(when(col(flag_temp) == 1, col(agg_col))),2).alias(f"VL_MAX_{agg_col}_{nome_flag_temp_corrigido}_POSCASH"))
            new_cols.append(round(min(when(col(flag_temp) == 1, col(agg_col))),2).alias(f"VL_MIN_{agg_col}_{nome_flag_temp_corrigido}_POSCASH"))
            new_cols.append(round(sum(when(col(flag_temp) == 1, col(agg_col))),2).alias(f"VL_SUM_{agg_col}_{nome_flag_temp_corrigido}_POSCASH"))
            new_cols.append(round(median(when(col(flag_temp) == 1, col(agg_col))),2).alias(f"VL_MD_{agg_col}_{nome_flag_temp_corrigido}_POSCASH"))


new_cols = tuple(new_cols)

temp03 = temp02.groupBy("SK_ID_PREV").agg(*new_cols).orderBy("SK_ID_PREV")

In [14]:
temp03 = temp03.repartition(1)
temp03.write.mode("overwrite").option("compression", "gzip").parquet("./VARS/POS_CASH_BALANCE/FL_TEMPORAL")

In [9]:
print('Quantidade de Vars. Criadas: ', len(temp03.columns))

Quantidade de Vars. Criadas:  49


#### 4.2 Usando Apenas Flags Categóricas

In [16]:
# new_cols = []

# for cat_flag in flags_categoricas:
#     nome_flag_cat_corrigido = cat_flag.replace('FL_CAT_','')
#     for agg_col in agg_cols:
#         if 'DPD' in agg_col:
#             new_cols.append(round(max(when(col(cat_flag) == 1, col(agg_col))),2).alias(f"QT_MAX_{agg_col}_{nome_flag_cat_corrigido}_POSCASH"))
#             new_cols.append(round(min(when(col(cat_flag) == 1, col(agg_col))),2).alias(f"QT_MIN_{agg_col}_{nome_flag_cat_corrigido}_POSCASH"))
#         else:
#             new_cols.append(round(max(when(col(cat_flag) == 1, col(agg_col))),2).alias(f"VL_MAX_{agg_col}_{nome_flag_cat_corrigido}_POSCASH"))
#             new_cols.append(round(min(when(col(cat_flag) == 1, col(agg_col))),2).alias(f"VL_MIN_{agg_col}_{nome_flag_cat_corrigido}_POSCASH"))
#             new_cols.append(round(sum(when(col(cat_flag) == 1, col(agg_col))),2).alias(f"VL_SUM_{agg_col}_{nome_flag_cat_corrigido}_POSCASH"))
#             new_cols.append(round(median(when(col(cat_flag) == 1, col(agg_col))),2).alias(f"VL_MD_{agg_col}_{nome_flag_cat_corrigido}_POSCASH"))

# new_cols = tuple(new_cols)

# temp04 = temp02.groupBy("SK_ID_PREV").agg(*new_cols).orderBy("SK_ID_PREV")

In [17]:
# temp04 = temp04.repartition(1)
# temp04.write.mode("overwrite").option("compression", "gzip").parquet("./VARS/POS_CASH_BALANCE/FL_CATEGORICAS")

In [17]:
# print('Quantidade de Vars. Criadas: ', len(temp03.columns))

Quantidade de Vars. Criadas:  49


### 5. Juntando as Tabelas

In [20]:
# Verificando a quantidade de linhas em cada tabela

temp03.count(), temp04.count()

(936325, 936325)

In [21]:
# Verificando a Quantidade de Colunas em cada tabela

len(temp03.columns), len(temp04.columns)

(37, 109)

* Após o Join, devemos esperar 936.325 linhas x 169 colunas

In [18]:
# pos_cash_agg = temp04.join(other= temp03, on= "SK_ID_PREV", how = 'left')

In [19]:
# Verificando as dimensões da tabela final

pos_cash_agg.count(), len(pos_cash_agg.columns)

(936325, 85)

* Salvando a Tabela Final

In [10]:
pos_cash_agg = temp03

pos_cash_agg = pos_cash_agg.repartition(1)

pos_cash_agg.write.mode("overwrite").parquet("./BASES_FEAT_ENG/POS_CASH_BALANCE_FEAT_ENG")